# Prefetch example

In [1]:
import sys
import feasst as fst

def mc(target_acceptance=0.2,
       tunable_param=0.185,
       density=0.85,
       cutoff=3,
       temperature=0.88,
       steps_per=int(1e6),
       ):
    box_length = 2*cutoff
    file_app = "_a" + str(target_acceptance) + "_r" + str(cutoff)
    num_particles=int(density*box_length**3)
    print('num_particles', num_particles)
    print('target_acceptance', target_acceptance)
    print('tunable_param', tunable_param)

    monte_carlo = fst.Prefetch(fst.args({"steps_per_check": "10000000"}))
    monte_carlo.activate_prefetch(False)
    monte_carlo.set(fst.lennard_jones(fst.args({"cubic_box_length": str(box_length)})))
    monte_carlo.set(fst.MakeThermoParams(fst.args({
        "beta": str(1./temperature),
        "chemical_potential": "1."})))
    monte_carlo.set(fst.MakeMetropolis())
    monte_carlo.add(fst.MakeTrialTranslate(fst.args({
        "weight": "1.",
        "tunable_param": str(tunable_param),
        "tunable_target_acceptance": str(target_acceptance),
        "tunable_percent_change": "0.01"})))
    fst.SeekNumParticles(num_particles).with_trial_add().run(monte_carlo)
    monte_carlo.add(fst.MakeLogAndMovie(fst.args({
        "steps_per" : str(steps_per),
        "file_name": "movie"+file_app,
        "clear_file": "true"})))
    monte_carlo.add(fst.MakeCheckEnergyAndTune(fst.args({"steps_per" : str(steps_per)})))
    monte_carlo.attempt(int(1e6)) #equilibrate
    monte_carlo.activate_prefetch(True)
    monte_carlo.add(fst.MakeMeanSquaredDisplacement(fst.args({
        "steps_per_update": "10000",
        "updates_per_origin": "1000",
        "file_name": "msd" + file_app + ".txt",
        "steps_per_write": str(int(1e5))})))
    monte_carlo.add(fst.MakeCPUTime(fst.args({
        "steps_per_update": str(steps_per),
        "steps_per_write": str(steps_per),
        "file_name": "cpu" + file_app + ".txt"})))
    monte_carlo.attempt(int(1e6))

mc()

num_particles 183
target_acceptance 0.2
tunable_param 0.185


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!